In [1]:
import sys
import os
import numpy as np
from datetime import datetime
import glob
#Get the path of the current file
current_path = os.getcwd()
# Add the evoman_framework folder to sys.path
sys.path.append(os.path.join(current_path, "evoman"))
# Now you can import contents from the evoman environment
from evoman.environment import Environment
from evoman.controller import Controller
# Initialize environment


pygame 2.6.0 (SDL 2.28.4, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


生成目录

In [2]:

# Set headless mode to improve running speed
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

experiment_name = "multi_island_optimization"
test_log_path = 'multi_island_optimization/test_logs/'
module_path = 'multi_island_optimization/models/' 
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

if not os.path.exists(test_log_path):
    os.makedirs(test_log_path)

if not os.path.exists(module_path):
    os.makedirs(module_path)


定义控制器

In [3]:
# Define Sigmoid activation function
def sigmoid_activation(x):
    return 1.0 / (1.0 + np.exp(-x))
# Custom controller with a simple neural network structure
class player_controller(Controller):
    def __init__(self, _n_hidden):
        self.n_hidden = [_n_hidden]  # Number of neurons in the hidden layer

    def set(self, controller, n_inputs):
        if self.n_hidden[0] > 0:
            # Extract weights and biases from controller parameters to build the neural network
            self.bias1 = controller[: self.n_hidden[0]].reshape(1, self.n_hidden[0])
            weights1_slice = n_inputs * self.n_hidden[0] + self.n_hidden[0]
            self.weights1 = controller[self.n_hidden[0] : weights1_slice].reshape(
                (n_inputs, self.n_hidden[0])
            )
            self.bias2 = controller[weights1_slice : weights1_slice + 5].reshape(1, 5)
            self.weights2 = controller[weights1_slice + 5 :].reshape(
                (self.n_hidden[0], 5)
            )

    def control(self, inputs, controller):
        # Normalize inputs
        inputs = (inputs - min(inputs)) / float((max(inputs) - min(inputs)))
        if self.n_hidden[0] > 0:
            output1 = sigmoid_activation(inputs.dot(self.weights1) + self.bias1)
            output = sigmoid_activation(output1.dot(self.weights2) + self.bias2)[0]
        else:
            bias = controller[:5].reshape(1, 5)
            weights = controller[5:].reshape((len(inputs), 5))
            output = sigmoid_activation(inputs.dot(weights) + bias)[0]

        # Determine player actions based on output
        left = 1 if output[0] > 0.5 else 0
        right = 1 if output[1] > 0.5 else 0
        jump = 1 if output[2] > 0.5 else 0
        shoot = 1 if output[3] > 0.5 else 0
        release = 1 if output[4] > 0.5 else 0

        return [left, right, jump, shoot, release]


# Initialize controller
player_controller_instance = player_controller(10)  # Example with 10 hidden neurons

定义函数

In [4]:
# Initialize population
def initialize_islands():
    islands = []
    for _ in range(n_islands):
        pop = np.random.uniform(dom_l, dom_u, (npop // n_islands, n_vars))
        fit_pop = evaluate(pop)
        islands.append((pop, fit_pop))
    return islands


def migrate(islands):
    num_migrants = int(migration_rate * (npop // n_islands))
    for i in range(n_islands):
        # Select migrants from island i
        migrants = islands[i][0][:num_migrants]
        # Migrate to the next island (circular migration)
        target_island = (i + 1) % n_islands
        islands[target_island][0][-num_migrants:] = migrants
    return islands

# Evolve each island's population independently
def evolve_islands(islands):
    for i in range(n_islands):
        pop, fit_pop = islands[i]
        elite_size = int(elitism_rate * len(pop))
        elite_pop, elite_fitness = elitism(pop, fit_pop, elite_size)

        # Crossover and mutation
        offspring = crossover_and_mutation(pop)
        fit_offspring = evaluate(offspring)

        # Combine elites and offspring
        pop = np.vstack((elite_pop, offspring[: len(pop) - elite_size]))
        fit_pop = np.hstack((elite_fitness, fit_offspring[: len(pop) - elite_size]))

        # Update the island's population and fitness
        islands[i] = (pop, fit_pop)
    return islands


# Run simulation and return fitness
def simulation(env, x):
    env.player_controller.set(x, n_inputs)  # Set controller parameters
    f, _, _, _ = env.play(pcont=x)
    return f


def simulation_test(env, x):
    env.player_controller.set(x, n_inputs)  # Set controller parameters
    f, player_life, enemy_life, _ = env.play(pcont=x)  # Return player and enemy life
    return f, player_life, enemy_life


# Evaluate population
def evaluate(x):
    return np.array([simulation(env, ind) for ind in x])

# Uniform crossover
def uniform_crossover(p1, p2):
    offspring = np.zeros_like(p1)
    for i in range(len(p1)):
        if np.random.rand() < 0.5:
            offspring[i] = p1[i]
        else:
            offspring[i] = p2[i]
    return offspring


# Fixed mutation
def mutation_operation(offspring):
    for i in range(len(offspring)):
        if np.random.uniform(0, 1) <= mutation:
            offspring[i] += np.random.normal(0, 1)
    return offspring


# Elitism selection
def elitism(pop, fit_pop, elite_size):
    elite_indices = np.argsort(fit_pop)[-elite_size:]
    elite_pop = pop[elite_indices]
    elite_fitness = fit_pop[elite_indices]
    return elite_pop, elite_fitness


# Crossover and mutation operations
def crossover_and_mutation(pop):
    total_offspring = np.zeros((0, n_vars))
    for p in range(0, pop.shape[0], 2):
        p1 = pop[np.random.randint(0, pop.shape[0])]
        p2 = pop[np.random.randint(0, pop.shape[0])]

        # Uniform crossover
        offspring = uniform_crossover(p1, p2)

        # Fixed mutation operation
        offspring = mutation_operation(offspring)

        # Fitness constraint
        offspring = np.clip(offspring, dom_l, dom_u)
        total_offspring = np.vstack((total_offspring, offspring))

    return total_offspring


# Save final solution
def save_final_solution(
    pop, fit_pop, experiment_name, enemies, npop, gens, mutation, elitism_rate
):
    best_index = np.argmax(fit_pop)
    best_solution = pop[best_index]
    best_fitness = fit_pop[best_index]

    # Get current timestamp
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Filename uses the best fitness
    filename = (
        f"{experiment_name}/models/MIModel_fitness{best_fitness:.6f}_npop{npop}_gens{gens}_"
        f"enemy{enemies}_mut{mutation:.2f}_elitism{elitism_rate:.2f}_{current_time}.txt"
    )

    # Save the best individual's controller weights and biases
    np.savetxt(filename, best_solution)

    # Print information about the saved file
    print(f"Best solution saved as {filename} with fitness: {best_fitness:.6f}")


# Load saved model
def load_best_solution(filepath):
    return np.loadtxt(filepath)


# Genetic algorithm evolution process
def evolve_population(pop, fit_pop):
    # Elitism selection
    elite_size = int(elitism_rate * npop)
    elite_pop, elite_fitness = elitism(pop, fit_pop, elite_size)

    # Crossover and mutation
    offspring = crossover_and_mutation(pop)
    fit_offspring = evaluate(offspring)

    # Combine elites and offspring
    pop = np.vstack((elite_pop, offspring[: npop - elite_size]))
    fit_pop = np.hstack((elite_fitness, fit_offspring[: npop - elite_size]))

    return pop, fit_pop


# Run evolution process
def run_evolution():
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{experiment_name}/test_logs/MI_log_{current_time}.txt"

    with open(filename, "a") as file:
        file.write(
            f"npop{npop}_gens{gens}_"
            f"enemy{env.enemies}_mut{mutation:.2f}_elitism{elitism_rate:.2f}_{current_time}\n"
        )
        file.write("gen best mean std\n")

        islands = initialize_islands()

        for generation in range(gens):
            # Evolve each island
            islands = evolve_islands(islands)

            # Perform migration every migration_interval generations
            if generation % migration_interval == 0 and generation > 0:
                islands = migrate(islands)

            # Record results (best, mean, std across all islands)
            all_fitness = np.hstack([fit_pop for _, fit_pop in islands])
            best = np.max(all_fitness)
            mean = np.mean(all_fitness)
            std = np.std(all_fitness)

            result = f"{generation} {best:.6f} {mean:.6f} {std:.6f}"
            print(result)
            file.write(result + "\n")

        # Find the best solution across all islands
        all_pop = np.vstack([pop for pop, _ in islands])
        all_fit_pop = np.hstack([fit_pop for _, fit_pop in islands])
        save_final_solution(all_pop, all_fit_pop, experiment_name, env.enemies, npop, gens, mutation, elitism_rate)



初始化环境

In [5]:
env = Environment(
    experiment_name=experiment_name,
    multiplemode="yes",
    playermode="ai",
    player_controller=player_controller_instance,
    enemymode="static",
    level=2,
    speed="fastest",
    visuals=False,
)



MESSAGE: Pygame initialized for simulation.


shm_open() failed: No such file or directory
ALSA lib conf.c:4028:(snd_config_hooks_call) Cannot open shared library libasound_module_conf_pulse.so (/home/taoyida/miniconda3/envs/EC/lib/alsa-lib/libasound_module_conf_pulse.so: cannot open shared object file: No such file or directory)
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM default


超参数

In [6]:
#set your enemy here
env.enemies=[5,6,7,8]

# Genetic algorithm parameters
n_inputs = 20  # Number of inputs to the controller
n_vars = (
    n_inputs * 10 + 10 + 10 * 5 + 5
)  # Total number of weights and biases in the controller
dom_u = 1
dom_l = -1
npop = 100  # Population size
gens = 100  # Total generations
mutation = 0.2  # Mutation rate
elitism_rate = 0.2  # Elitism rate
migration_interval = 3  # Every 10 generations
migration_rate = 0.1  # 10% of the population migrates
n_islands = 4

In [22]:
for i in range(10):
    print(f"Running evolution iteration {i+1}")
    run_evolution()

Running evolution iteration 1
0 70.082386 -1.593997 11.302658
1 70.082386 0.430024 13.240395
2 70.082386 6.105035 18.086122
3 70.082386 2.089879 18.824147
4 70.082386 8.719492 20.121996
5 70.082386 6.790683 18.212904
6 70.082386 11.714733 21.899189
7 70.082386 9.025667 18.840311
8 70.082386 9.038566 22.205302
9 70.082386 5.237773 19.398053
10 70.082386 10.645740 21.788987
11 70.082386 7.937276 19.716723
12 70.082386 12.292664 22.525674
13 70.082386 6.913545 20.031648
14 70.082386 19.911041 23.658228
15 70.082386 12.624948 22.698236
16 70.082386 13.565896 22.222532
17 70.082386 12.650189 21.761777
18 70.082386 14.719738 26.268951
19 70.082386 12.344091 24.118773
20 70.082386 9.021348 23.632436
21 70.082386 9.008575 21.368408
22 70.082386 12.085511 23.781524
23 70.082386 8.453811 21.505229
24 70.082386 8.867654 23.699381
25 70.082386 6.706988 20.646401
26 70.082386 12.247742 24.211539
27 70.082386 10.668964 22.284892
28 70.082386 12.805706 24.357764
29 70.082386 10.563454 20.980938
30 70

测试

In [7]:
def test_loaded_model():
    # Hardcoded model file path
    filepath = f"{experiment_name}/models/MIModel_fitness11.762481_npop100_gens5_enemy[1, 2, 3, 4]_mut0.20_elitism0.20_20241015_154502.txt"

    fitness_scores = []
    gains = []  # Used to store individual_gain for each run

    # Load the saved model
    solution = load_best_solution(filepath)

    # Run 5 times
    for i in range(5):
        fitness, player_life, enemy_life = simulation_test(env, solution)
        fitness_scores.append(fitness)

        # Calculate individual_gain
        individual_gain = player_life - enemy_life
        gains.append(individual_gain)

        print(f"Run {i+1} fitness: {fitness}, individual_gain: {individual_gain}")

    # Calculate the average fitness
    average_fitness = np.mean(fitness_scores)
    average_gain = np.mean(gains)

    print(f"\nAverage fitness over 5 runs: {average_fitness}")
    print(f"Average individual gain over 5 runs: {average_gain}")


# # Run test
# test_loaded_model()


def evaluate_models(experiment_name, enemy):
    # 获取所有模型文件
    model_files = glob.glob(f"{experiment_name}/models/E{enemy}/*.txt")

    all_models_gains = {}

    for filepath in model_files:
        fitness_scores = []
        gains = []  # 用于存储每次运行的 individual_gain

        # 加载保存的模型
        solution = load_best_solution(filepath)

        # 运行一次
        fitness, player_life, enemy_life = simulation_test(env, solution)
        fitness_scores.append(fitness)

        # 计算 individual_gain
        individual_gain = player_life - enemy_life
        gains.append(individual_gain)

        print(
            f"Model: {filepath}, fitness: {fitness}, individual_gain: {individual_gain}"
        )

        # 计算平均 fitness
        average_fitness = np.mean(fitness_scores)
        average_gain = np.mean(gains)

        print(f"\nModel: {filepath}, Average fitness: {average_fitness}")
        print(
            f"Model: {filepath}, Average individual gain: {average_gain}\n"
        )

        # 保存每个模型的 gains
        all_models_gains[filepath] = gains

    # 打印所有模型的 gains
    for model, gains in all_models_gains.items():
        print(f"Model: {model}, Gains: {gains}")

    # 将所有模型的 enemy 和 gains 保存到 CSV 文件
    all_models_info_path = f"{experiment_name}/models/all_models_{experiment_name}_enemy_{enemy}.csv"
    with open(all_models_info_path, "w") as file:
        file.write("Enemy,Gain\n")
        for model, gains in all_models_gains.items():
            for gain in gains:
                file.write(f"{enemy},{gain}\n")
    print(f"All models info saved to {all_models_info_path}")

In [9]:
env.enemies = [1, 2, 3, 4]
evaluate_models(experiment_name, 5678)

Model: multi_island_optimization/models/E5678/MIModel_fitness78.965877_npop100_gens100_enemy[5, 6, 7, 8]_mut0.20_elitism0.20_20241016_153947.txt, fitness: -5.927346918257382, individual_gain: -100.0

Model: multi_island_optimization/models/E5678/MIModel_fitness78.965877_npop100_gens100_enemy[5, 6, 7, 8]_mut0.20_elitism0.20_20241016_153947.txt, Average fitness: -5.927346918257382
Model: multi_island_optimization/models/E5678/MIModel_fitness78.965877_npop100_gens100_enemy[5, 6, 7, 8]_mut0.20_elitism0.20_20241016_153947.txt, Average individual gain: -100.0

Model: multi_island_optimization/models/E5678/MIModel_fitness78.829506_npop100_gens100_enemy[5, 6, 7, 8]_mut0.20_elitism0.20_20241016_154107.txt, fitness: 4.712395354922464, individual_gain: -22.191031145072035

Model: multi_island_optimization/models/E5678/MIModel_fitness78.829506_npop100_gens100_enemy[5, 6, 7, 8]_mut0.20_elitism0.20_20241016_154107.txt, Average fitness: 4.712395354922464
Model: multi_island_optimization/models/E5678/